In [1]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=20)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:   tcp://10.32.16.73:45557
distributed.scheduler - INFO -   dashboard at:                     :8787


In [2]:
import zarr
import netCDF4
import pytide
import xarray as xr

In [3]:
import gcsfs
fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)


In [4]:
%%time
zmap = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSH-1h')
ds = xr.open_zarr(zmap)
ds_sorted=ds.sortby('time_counter')
ssh=ds_sorted['sossheig']
time=ds_sorted['time_counter']

CPU times: user 1.63 s, sys: 362 ms, total: 2 s
Wall time: 8.73 s


In [5]:
t=time.values

In [6]:
wt = pytide.WaveTable()
wt = pytide.WaveTable(["M2", "S2", "N2", "O1", "K1"])

In [7]:
%%time
f, vu = wt.compute_nodal_modulations(t)

CPU times: user 10.1 ms, sys: 0 ns, total: 10.1 ms
Wall time: 8.98 ms


In [ ]:
%%time
w = wt.harmonic_analysis(ssh.values, f, vu)

In [ ]:
%%time
hp = wt.tide_from_tide_series(t,w)

In [ ]:
cluster.close()